In [42]:
import pandas as pd
import numpy as np
import os
import datetime

In [43]:
def edit_column_date(frame,index):
    #Edits the date format of columns of dataframes
    #index: index of the first column of dates + 1
    i = 0
    for col in frame:
        i += 1
        if i >= index:
            new_d = date_format(col)
            frame = frame.rename(columns={col : new_d})
    return frame

In [44]:
def sort_dates(frame,index):
    #Sorts the columns by date of a frame with many nonconsecutive dates (several factors per date)
    Beg = list(frame.columns[:index]) #First four entries
    End = list(np.sort(np.array(frame.columns[index:]))) #Every Date Sorted
    cols = list(Beg + End) #Ordered Columns

    frame = frame[cols]
    return frame

In [45]:
def date_format(date):
    d = datetime.datetime.strptime(date, '%Y-%m-%d')
    return datetime.date.strftime(d, "%m/%d/%y")

In [46]:
#Loading in mobility data
DL_us_m50 = pd.read_csv('../../../../data/us/mobility/DL-us-m50.csv', encoding='latin1')
DL_us_m50_index = pd.read_csv('../../../../data/us/mobility/DL-us-m50_index.csv', encoding='latin1')
DL_us_samples = pd.read_csv('../../../../data/us/mobility/DL-us-samples.csv')

In [47]:
#Cleaning the datasets
DL_us_m50 = edit_column_date(DL_us_m50,6)
DL_us_m50_index = edit_column_date(DL_us_m50_index,6)
DL_us_samples = edit_column_date(DL_us_samples,6)

DL_us_m50 = DL_us_m50.drop(columns=['country_code','admin_level','admin1','admin2'])
DL_us_m50_index = DL_us_m50_index.drop(columns=['country_code','admin_level','admin1','admin2'])
DL_us_samples = DL_us_samples.drop(columns=['country_code','admin_level','admin1','admin2'])

In [48]:
#Separating data into county info

DL_us_m50_County = DL_us_m50[DL_us_m50.fips >= 1000]
DL_us_m50_index_County = DL_us_m50_index[DL_us_m50_index.fips >= 1000]
DL_us_samples_County = DL_us_samples[DL_us_samples.fips >= 1000]

In [49]:
#merging the 3 datasets together
Mobility_County = pd.merge(DL_us_m50_County, DL_us_m50_index_County, \
                    left_on='fips', right_on='fips', suffixes=('_M_m50', ''), sort=True)
Mobility_County = pd.merge(Mobility_County, DL_us_samples_County, \
                    left_on='fips', right_on='fips', suffixes=('_M_idx', '_M_samples'), sort=True)
Mobility_County = Mobility_County[Mobility_County.fips >= -1]
Mobility_County.columns = Mobility_County.columns.str.replace('fips','FIPS')
#saving datasets with 3 values not consecutive and then consecutive
Mobility_County_Nonconsecutive = Mobility_County
Mobility_County_Consecutive = sort_dates(Mobility_County,1)
#MAking FIPS the main index
Mobility_County_Consecutive = Mobility_County_Consecutive.set_index('FIPS')
Mobility_County_Nonconsecutive = Mobility_County_Nonconsecutive.set_index('FIPS')

Mobility_County_Consecutive.to_csv('Mobility_County_Consecutive.csv')
Mobility_County_Nonconsecutive.to_csv('Mobility_County_Nonconsecutive.csv')

In [50]:
#New Google Mobility Data, must be processed

google_mobility = pd.read_csv('../../../../data/google_mobility/mobility_report_US.csv', encoding='latin1')
#Taking only county data
google_mobility_county = google_mobility[google_mobility['Region'] != 'Total']

#Key to map counties to FIPS, and states to state abbreviations
Key =  pd.read_csv('Key.csv').sort_values(by=['FIPS'])
State_Abv = pd.read_csv('../State_Abbrev.csv')
State_Abv = np.array(State_Abv)
#Dictionary from state names to state initials
State_Dict = dict((rows[0],rows[2]) for rows in State_Abv)

#Changing the state column of google mobility to its abbreviation code
google_mobility_county = google_mobility_county.replace({'State': State_Dict})

#Creating a location column, to make the google mobility locations unique
google_mobility_county['loc'] = google_mobility_county.Region.astype(str).str.cat(google_mobility_county.State.astype(str), sep=', ')
Key['loc'] = Key.COUNTY.astype(str).str.cat(Key.ST.astype(str), sep=', ')

#New google county mobility data, with fips codes attached
google_county = pd.merge(google_mobility_county, Key, \
                    left_on='loc', right_on='loc', sort=True)
#removing unecessary columns
google_county = google_county.drop(columns=['State','Region','ST','COUNTY','loc'])


In [51]:
#Splitting up this google county into its components to rejoin it later
google_residential = google_county.pivot(index='FIPS', columns='Date', values=['Residential'])
google_residential.to_csv('google_residential.csv')
#Reading in split up component and the resetting the header values
google_residential = pd.read_csv('google_residential.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_residential = edit_column_date(google_residential,2)

google_workplaces = google_county.pivot(index='FIPS', columns='Date', values=['Workplaces'])
google_workplaces.to_csv('google_workplaces.csv')
google_workplaces = pd.read_csv('google_workplaces.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_workplaces = edit_column_date(google_workplaces,2)

google_transit = google_county.pivot(index='FIPS', columns='Date', values=['Transit stations'])
google_transit.to_csv('google_transit.csv')
google_transit = pd.read_csv('google_transit.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_transit = edit_column_date(google_transit,2)

google_parks = google_county.pivot(index='FIPS', columns='Date', values=['Parks'])
google_parks.to_csv('google_parks.csv')
google_parks = pd.read_csv('google_parks.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_parks = edit_column_date(google_parks,2)

google_grocery = google_county.pivot(index='FIPS', columns='Date', values=['Grocery & pharmacy'])
google_grocery.to_csv('google_grocery.csv')
google_grocery = pd.read_csv('google_grocery.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_grocery = edit_column_date(google_grocery,2)

google_retail = google_county.pivot(index='FIPS', columns='Date', values=['Retail & recreation'])
google_retail.to_csv('google_retail.csv')
google_retail = pd.read_csv('google_retail.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_retail = edit_column_date(google_retail,2)

#Merging the data back together
google_county = pd.merge(google_residential, google_workplaces, \
                    left_on='FIPS', right_on='FIPS', suffixes=('_residential', ''))
google_county = pd.merge(google_county, google_transit, \
                    left_on='FIPS', right_on='FIPS', suffixes=('_workplaces', ''))
google_county = pd.merge(google_county, google_parks, \
                    left_on='FIPS', right_on='FIPS', suffixes=('_transit', ''))
google_county = pd.merge(google_county, google_grocery, \
                    left_on='FIPS', right_on='FIPS', suffixes=('_parks', ''))
google_county = pd.merge(google_county, google_retail, \
                    left_on='FIPS', right_on='FIPS', suffixes=('_grocery', 'retail'))


In [52]:
#saving google dataset with each component either not consecutive and then consecutive
google_county_Nonconsecutive = google_county
google_county_Consecutive = sort_dates(google_county,1)

#MAking FIPS the main index
google_county_Consecutive = google_county_Consecutive.set_index('FIPS')
google_county_Nonconsecutive = google_county_Nonconsecutive.set_index('FIPS')

google_county_Consecutive.to_csv('google_county_Consecutive.csv')
google_county_Nonconsecutive.to_csv('google_county_Nonconsecutive.csv')

In [53]:
print('google_mobility: ' + str(len(google_mobility)))
print('Key: ' + str(len(Key)))
print('google_mobility_county: ' + str(len(google_mobility_county)))
print('google_county_Consecutive: ' + str(len(google_county_Consecutive)))


google_mobility: 5728
Key: 3246
google_mobility_county: 5626
google_county_Consecutive: 2759


In [54]:
print('DL_us_m50_County: ' + str(len(DL_us_m50_County)))
print('DL_us_m50_index_County: ' + str(len(DL_us_m50_index_County)))
print('DL_us_samples_County: ' + str(len(DL_us_samples_County)))
print('Mobility_County_Consecutive: ' + str(len(Mobility_County_Consecutive)))

DL_us_m50_County: 2670
DL_us_m50_index_County: 2670
DL_us_samples_County: 2670
Mobility_County_Consecutive: 2676


In [55]:
Mobility_County_Consecutive.head()

,03/01/20_M_idx,03/01/20_M_m50,03/01/20_M_samples,03/02/20_M_idx,03/02/20_M_m50,03/02/20_M_samples,03/03/20_M_idx,03/03/20_M_m50,03/03/20_M_samples,03/04/20_M_idx,...,04/06/20_M_samples,04/07/20_M_idx,04/07/20_M_m50,04/07/20_M_samples,04/08/20_M_idx,04/08/20_M_m50,04/08/20_M_samples,04/09/20_M_idx,04/09/20_M_m50,04/09/20_M_samples
FIPS,,,,,,,,,,,,,,,,,,,,,
1001.0,49.0,7.194,1703.0,100.0,14.587,1829.0,95.0,13.865,1840.0,95.0,...,1719.0,33.0,4.851,1696.0,32.0,4.792,1685.0,38.0,5.662,1696.0
1003.0,81.0,9.780,7067.0,100.0,12.042,7136.0,95.0,11.481,7220.0,90.0,...,6397.0,39.0,4.726,6463.0,41.0,5.042,6423.0,46.0,5.588,6477.0
1005.0,90.0,8.348,546.0,107.0,10.004,569.0,100.0,9.267,545.0,70.0,...,508.0,47.0,4.406,503.0,47.0,4.388,510.0,64.0,5.946,524.0
1007.0,53.0,13.008,512.0,95.0,23.076,574.0,100.0,24.164,588.0,94.0,...,513.0,29.0,7.141,505.0,43.0,10.532,495.0,41.0,9.952,508.0
1009.0,68.0,15.963,1495.0,96.0,22.456,1608.0,100.0,23.222,1615.0,99.0,...,1500.0,41.0,9.551,1503.0,43.0,10.065,1499.0,49.0,11.477,1499.0


In [56]:
google_county_Consecutive.head()

,03/29/20_grocery,03/29/20_parks,03/29/20_residential,03/29/20_transit,03/29/20_workplaces,03/29/20retail,04/05/20_grocery,04/05/20_parks,04/05/20_residential,04/05/20_transit,04/05/20_workplaces,04/05/20retail
FIPS,,,,,,,,,,,,
1001,-8.0,-14.0,16.0,NaN,-35.0,-42.0,-8.0,-14.0,17.0,NaN,-38.0,-53.0
1003,-19.0,-27.0,9.0,-24.0,-32.0,-47.0,-27.0,-44.0,11.0,-39.0,-35.0,-55.0
1005,-20.0,NaN,NaN,NaN,-23.0,-27.0,-20.0,NaN,NaN,NaN,-30.0,-40.0
1007,-10.0,NaN,NaN,NaN,-32.0,-25.0,-15.0,NaN,NaN,NaN,-35.0,-42.0
1009,-2.0,NaN,15.0,NaN,-29.0,-36.0,-17.0,NaN,14.0,NaN,-33.0,-49.0
